<a href="https://colab.research.google.com/github/isaacpycad/Spark_colab/blob/main/DF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd
!wget https://mirror.olnevhost.net/pub/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

/content
--2021-01-30 03:48:52--  https://mirror.olnevhost.net/pub/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving mirror.olnevhost.net (mirror.olnevhost.net)... 188.165.227.148
Connecting to mirror.olnevhost.net (mirror.olnevhost.net)|188.165.227.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  13.7MB/s    in 24s     

2021-01-30 03:49:17 (8.62 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [219929956/219929956]



In [2]:
!tar -xvzf spark-3.0.1-bin-hadoop2.7.tgz

spark-3.0.1-bin-hadoop2.7/
spark-3.0.1-bin-hadoop2.7/RELEASE
spark-3.0.1-bin-hadoop2.7/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/
spark-3.0.1-bin-hadoop2.7/examples/src/main/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/FPGrowthExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/GBTExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/ALSExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/KMeansExample.scala
spark-3.0.1-bin-hadoop2.7/example

In [3]:
!ls /content/spark-3.0.1-bin-hadoop2.7

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [4]:
!pip install findspark

In [5]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
import findspark
findspark.init()

In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [8]:
from pyspark.sql import SparkSession

In [10]:
spark=SparkSession.builder.appName('basics').getOrCreate()

In [11]:
from google.colab import files
files.upload()

Saving people.json to people.json


{'people.json': b'{"name":"Michael"}\n{"name":"Andy", "age":30}\n{"name":"Justin", "age":19}\n'}

In [12]:
df=spark.read.json("people.json")

In [13]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [15]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [16]:
#this is an attribute not a meathod thats why ends without ()
df.columns

['age', 'name']

In [17]:
#statistical summanry of numeric data of our columns
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [18]:
from pyspark.sql.types import StructField,StructType,IntegerType,StringType

In [19]:
data_schema=[StructField("age",IntegerType(),True),
             StructField("name",StringType(),True)]

In [20]:
final_struct=StructType(fields=data_schema)

In [21]:
df=spark.read.json("people.json",schema=final_struct)

In [23]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [24]:
#as spark evolves, spark it self infers the schema right way but in some cases we need to redefine the schema thats why we took previous steps.

In [25]:
#Following is hot to select a dataframe from a dataframe.


In [27]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [28]:
#lets select rows from the dataframe

In [29]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [30]:
df.head(2)[0]

Row(age=None, name='Michael')

In [31]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [32]:
##add column with "withcolumn"

In [34]:
df.withColumn("new_age",df['age']).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     30|
|  19| Justin|     19|
+----+-------+-------+



In [35]:
df.withColumn("double_age",df["age"]*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [36]:
#use direct SQL in spark

In [37]:
df.createOrReplaceTempView("ppl")

In [46]:
sql_result=spark.sql("select * from ppl where age=30")

In [47]:
sql_result.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

